In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

### Importazione dataframe

In [2]:
df1 = pd.read_csv('C:/Users/HP/Desktop/PLC_gualchierani/monitorconi_23mar_to_29mar.csv')
df2 = pd.read_csv('C:/Users/HP/Desktop/PLC_gualchierani/monitorconi_30mar_to_4apr.csv')
df3 = pd.read_csv('C:/Users/HP/Desktop/PLC_gualchierani/monitorconi_5apr_to_11apr.csv')
df4 = pd.read_csv('C:/Users/HP/Desktop/PLC_gualchierani/monitorconi_12apr_to_18apr.csv')
df5 = pd.read_csv('C:/Users/HP/Desktop/PLC_gualchierani/monitorconi_19apr_to_25apr.csv')

In [3]:
df1["measured_at"] = pd.to_datetime( df1["measured_at"] )
df2["measured_at"] = pd.to_datetime( df2["measured_at"] )
df3["measured_at"] = pd.to_datetime( df3["measured_at"] )
df4["measured_at"] = pd.to_datetime( df4["measured_at"] )
df5["measured_at"] = pd.to_datetime( df5["measured_at"] )

In [4]:
mask_30mar = np.asarray( df1[ df1['measured_at'].dt.day == 30].index )
df1.drop( mask_30mar, inplace = True )

mask_29mar = np.asarray( df2[ df2['measured_at'].dt.day == 29].index )
df2.drop( mask_29mar, inplace = True )

mask_5apr = np.asarray( df2[ df2['measured_at'].dt.day == 5].index )
df2.drop( mask_5apr, inplace = True )

mask_4apr = np.asarray( df3[ df3['measured_at'].dt.day == 4].index )
df3.drop( mask_4apr, inplace = True )

mask_12apr = np.asarray( df3[ df3['measured_at'].dt.day == 12].index )
df3.drop( mask_12apr, inplace = True )

mask_4apr = np.asarray( df4[ df4['measured_at'].dt.day == 11].index )
df4.drop( mask_4apr, inplace = True )

mask_12apr = np.asarray( df4[ df4['measured_at'].dt.day == 19].index )
df4.drop( mask_12apr, inplace = True )

mask_4apr = np.asarray( df5[ df5['measured_at'].dt.day == 18].index )
df5.drop( mask_4apr, inplace = True )

mask_12apr = np.asarray( df5[ df5['measured_at'].dt.day == 26].index )
df5.drop( mask_12apr, inplace = True )

df = pd.concat([df1, df2, df3, df4, df5])

In [5]:
df.drop(['id', 'tipologia_lavorazione', 'cicli_da_eseguire',
        'cicli_eseguiti', 'tempo_miscelazione_eseguito',
        'stato_tramoggia', 'tara_cono', 'lotto',
        'created_at', 'updated_at'], axis=1, inplace=True)

In [6]:
mask_no_cono = np.asarray( df[ df['id_cono'] > 24 ].index )
df.drop( mask_no_cono, inplace = True )

In [7]:
len(df)

942915

In [8]:
df.dropna(inplace=True)

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 942915 entries, 0 to 528442
Data columns (total 9 columns):
 #   Column                            Non-Null Count   Dtype         
---  ------                            --------------   -----         
 0   id_cono                           942915 non-null  int64         
 1   ubicazione                        942915 non-null  int64         
 2   stato                             942915 non-null  int64         
 3   postazione_pesatura_destinazione  942915 non-null  int64         
 4   miscelatore_destinazione          942915 non-null  int64         
 5   estrusore_destinazione            942915 non-null  int64         
 6   tempo_miscelazione_richiesto      942915 non-null  int64         
 7   destinazione_ritorno              942915 non-null  int64         
 8   measured_at                       942915 non-null  datetime64[ns]
dtypes: datetime64[ns](1), int64(8)
memory usage: 71.9 MB


### Preparazione dati di supporto

In [10]:
buffer = [n for n in range(21, 29)]
buffer.extend([n for n in range(31, 39)])

legenda_gualchierani = {
    'fuori_impianto': [0],
    'ing_zona_misc': [1],
    'catenaria_ing_misc': [45, 46],
    'navetta_serv_misc': [43],
    'pesatura': [41, 42],
    'miscelatore': [2, 8],
    'cat_usc_pieni': [3],
    'nav_serv_carr_pieni': [4],
    'nav_serv_carr_vuoti': [5],
    'buffer': buffer,
    'carroponte_1': [6],
    'carroponte_2': [7],
    'estrusore': [n for n in range(11, 19)],
    'handling_post_est': [6, 5, 4],
    'catenaria_vuoti': [47, 48],
    'catenaria_uscita_vuoti': [49],
    'pinza_vuoti': [50],
    'catenaria_parcheggi': [n for n in range(51, 58)],
    'nav_serv_parcheggi': [44],
    'manuale': [68, 70]
}

dict_states = {
    0: 'Not_def',
    1: 'V_per_park',
    2: 'P_da_misc',
    3: 'P_da_est',
    4: 'P_remove',
    5: 'P_recheck',
    6: 'P_per_park',
    7: 'V_da_pes',
    8: 'V_remove',
    9: 'V_lavaggio'
}

starter = ['V_da_pes/nav_serv_carr_pieni', 'V_da_pes/nav_serv_carr_vuoti',
'V_da_pes/catenaria_vuoti', 'V_da_pes/catenaria_parcheggi',
'V_da_pes/nav_serv_parcheggi', 'V_da_pes/catenaria_ing_misc',
'V_da_pes/ing_zona_misc']

finisher = ['P_da_est/estrusore', 'Not_def/fuori_impianto']

finisher2 = ['V_per_park/estrusore', 'V_da_pes/estrusore', 'Not_def/fuori_impianto']

### Preparazione funzioni

In [11]:
#  questa funzione estrae dal df il primo record in ordine temporale per ogni ubicazione

def create_df_t(df, cono_in_esame):
    mask = df['id_cono'] == cono_in_esame
    df1 = df[mask]

    ubicazione = []
    stato = []
    timestamp = []
    for i in range(len(df1) - 1):
        if i == 0:
            ubicazione.append(df1.iloc[i, 1])
            stato.append(df1.iloc[i, 2])
            timestamp.append(df1.iloc[i, 8])
        elif df1.iloc[i, 1] == df1.iloc[i - 1, 1]:
            continue
        else:
            ubicazione.append(df1.iloc[i, 1])
            stato.append(df1.iloc[i, 2])
            timestamp.append(df1.iloc[i, 8])

#  estrae i valori dalla memoria del PLC li traduco in parole
    for i in range(len(ubicazione)):
        for k,v in legenda_gualchierani.items():
            if ubicazione[i] in v:
                ubicazione[i] = k

    for i in range(len(stato)):
        stato[i] = dict_states[stato[i]]

#  crea il dataframe che sarà input alla funzione di individuazione della tracce
    df_t = pd.DataFrame(list(zip(stato, ubicazione, timestamp)), columns =['stato', 'ubicazione', 'timestamp'])
    df_t.sort_values('timestamp', inplace=True)
    df_t['ID'] = 0
    df_t['CRC'] = 0
    df_t['rif'] = (df_t.stato.astype('str') + '/' + df_t.ubicazione.astype('str'))
    return(df_t)

In [12]:
#  questa funzione estrae dal df il primo record in ordine temporale per ogni ubicazione

def create_df_t2(df, cono_in_esame):
    mask = df['id_cono'] == cono_in_esame
    df1 = df[mask]

    ubicazione = []
    stato = []
    timestamp = []
    for i in range(len(df1) - 1):
        if i == 0:
            ubicazione.append(df1.iloc[i, 1])
            stato.append(df1.iloc[i, 2])
            timestamp.append(df1.iloc[i, 8])
        if df1.iloc[i, 1] in legenda_gualchierani['estrusore']:
            if ((df1.iloc[i, 1] == df1.iloc[i - 1, 1] and df1.iloc[i, 1] != df1.iloc[i + 1, 1])
                    or df1.iloc[i, 1] != df1.iloc[i - 1, 1]):
                ubicazione.append(df1.iloc[i, 1])
                stato.append(df1.iloc[i, 2])
                timestamp.append(df1.iloc[i, 8])
        elif df1.iloc[i, 1] == df1.iloc[i - 1, 1]:
            continue
        else:
            ubicazione.append(df1.iloc[i, 1])
            stato.append(df1.iloc[i, 2])
            timestamp.append(df1.iloc[i, 8])

#  estrae i valori dalla memoria del PLC li traduco in parole
    for i in range(len(ubicazione)):
        for k,v in legenda_gualchierani.items():
            if ubicazione[i] in v:
                ubicazione[i] = k

    for i in range(len(stato)):
        stato[i] = dict_states[stato[i]]

#  crea il dataframe che sarà input alla funzione di individuazione della tracce
    df_t = pd.DataFrame(list(zip(stato, ubicazione, timestamp)), columns =['stato', 'ubicazione', 'timestamp'])
    df_t.sort_values('timestamp', inplace=True)
    df_t['ID'] = 0
    df_t['CRC'] = 0
    df_t['rif'] = (df_t.stato.astype('str') + '/' + df_t.ubicazione.astype('str'))
    df_t.drop_duplicates(inplace=True)
    return(df_t)

In [13]:
#  questa funzione individua delle potenziali tracce di processo
#  prende in input una lista di possibili punti di inizio ed una di possibili punti di fine

def create_traces(df_t, cono_in_esame, starter, finisher):
    global trace
    df_t2 = df_t.copy()
    cont_related_counter = 0
    bol_end = False
    bol_start = False
    for idx in df_t2.index:
        global trace
        if df_t2.loc[idx, 'rif'] in starter and not bol_start:
            bol_end = False
            bol_start = True
            trace += 1
            cont_related_counter += 1
            df_t2.loc[idx, 'ID'] = trace
            df_t2.loc[idx, 'CRC'] = cont_related_counter
        elif df_t2.loc[idx, 'rif'] == 'V_da_pes/pesatura' and not bol_end:
            df_t2.loc[idx, 'ID'] = trace
            df_t2.loc[idx, 'CRC'] = cont_related_counter
            bol_end = True
            bol_start = False
        elif not bol_end:
            df_t2.loc[idx, 'ID'] = trace
            df_t2.loc[idx, 'CRC'] = cont_related_counter
    
    #  elimina dal df tutti i record che non sono stati attribuiti a nessuna traccia
    mask_no_trace = np.asarray( df_t2[ df_t2['ID'] == 0 ].index )
    df_t2.drop( mask_no_trace, inplace = True )
    
    #  qualche trick per limitare i doppioni
    trace = trace + 1
    df_t2['ID_unique'] = (df_t2.CRC.astype('str') + 'C' + str(cono_in_esame) + df_t2.ID.astype('str'))
    return(df_t2)

In [14]:
def create_traces3(df_t, cono_in_esame, finisher):
    global trace
    df_t2 = df_t.copy()
    cont_related_counter = 0
    bol_end = False
    bol_start = False
    for idx in df_t2.index:
        global trace
        if df_t2.loc[idx, 'rif'] == 'V_da_pes/pesatura' and not bol_start:
            bol_end = False
            bol_start = True
            trace += 1
            cont_related_counter += 1
            df_t2.loc[idx, 'ID'] = trace
            df_t2.loc[idx, 'CRC'] = cont_related_counter
        elif df_t2.loc[idx, 'rif'] in finisher and not bol_end:
            df_t2.loc[idx, 'ID'] = trace
            df_t2.loc[idx, 'CRC'] = cont_related_counter
            bol_end = True
            bol_start = False
        elif not bol_end:
            df_t2.loc[idx, 'ID'] = trace
            df_t2.loc[idx, 'CRC'] = cont_related_counter
    
    mask_no_trace = np.asarray( df_t2[ df_t2['ID'] == 0 ].index )
    df_t2.drop( mask_no_trace, inplace = True )
    
    #  qualche trick per limitare i doppioni
    trace = trace + 10
    df_t2['ID_unique'] = (df_t2.CRC.astype('str') + 'C' + str(cono_in_esame) + df_t2.ID.astype('str'))
    if df_t2['CRC'].isnull().values.any():
        print('CI SONO DEI NULL SECONDA FUNZIONE  CONO  ', cono_in_esame)
    return(df_t2)

### Calcolo tracce

In [ ]:
trace = 0
for c in range(1, 25):
    global trace
    start = datetime.now()
    df_pre_trace = create_df_t2(df, c)
    if c == 1:
        df_traces = create_traces3(df_pre_trace, c, finisher2)
    else:
        df_traces = pd.concat([df_traces, create_traces3(df_pre_trace, c, finisher2)])
    end = datetime.now()
    print('TEMPO ELABORAZIONE CONO {} {}'.format(c, end-start))

In [ ]:
df_traces.to_csv('C:/Users/HP/Desktop/df_traces_from_weigh.csv')

In [ ]:
len(df_traces['ID_unique'].unique())

In [ ]:
df_traces